In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

### Creación de funciones auxiliares

In [5]:
def discretize(df, column):
    q1 = df_final[column].quantile(.25)
    q2 = df_final[column].quantile(.5)
    q3 = df_final[column].quantile(.75)
    df[column+"_in_q1"] = 0
    df[column+"_in_q2"] = 0
    df[column+"_in_q3"] = 0
    df[column+"_in_q4"] = 0
    df[column+"_in_q1"] = list(map(lambda x: 1 if x <=q1 else 0, df[column]))
    df[column+"_in_q2"] = list(map(lambda x: 1 if (q1 < x and x <=q2) else 0, df[column]))
    df[column+"_in_q3"] = list(map(lambda x: 1 if (q2 < x and x <=q3) else 0, df[column]))
    df[column+"_in_q4"] = list(map(lambda x: 1 if x>q3 else 0, df[column]))
    df.drop(column, axis=1, inplace=True)
    return df

In [6]:
def is_object(df,column,object_name):
    df["is_"+str(object_name)] = list(map(lambda x: 1 if x == object_name else 0, df[column]))
    return df
    

## Cargamos los datasets para construir el dataset final

In [7]:
df_first = pd.read_csv('../../data_aa/first_part_dataset.csv')

df_second = pd.read_csv('../../data_aa/second_part_dataset.csv')

df_thrid = pd.read_csv('../../data_aa/thrid_part_dataset.csv')

In [8]:
df_first.columns.values

array(['Unnamed: 0', 'Android', 'BlackBerry', 'Chrome', 'Computer',
       'FreeBSD', 'Linux', 'Mac', 'Other', 'Smartphone', 'Tablet',
       'Tizen', 'Ubuntu', 'Unknown', 'Windows', 'ad campaign hit',
       'brand listing', 'checkout', 'conversion', 'generic listing',
       'iOS', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'target', 'viewed product', 'visited site',
       'person_id'], dtype=object)

In [9]:
df_first = df_first.rename(columns={'person_id': 'person'})

df_first = df_first.drop(columns=['Unnamed: 0'])

In [10]:
df_second.columns.values

array(['Unnamed: 0', 'person', 'diff_time_days_minnone',
       'diff_time_daysnone', 'diff_time_dayscheckout',
       'diff_time_daysconversion', 'diff_time_daysbrand listing',
       'diff_time_dayssearch engine hit', 'diff_time_start_end', 'model',
       'number_visits_model'], dtype=object)

In [11]:
df_second = df_second.drop(columns=['Unnamed: 0'])

In [12]:
df_thrid.columns.values

array(['Unnamed: 0', 'person', 'COUNT(users_logs)', 'avg_time_events'],
      dtype=object)

In [13]:
df_thrid = df_thrid.drop(columns=['Unnamed: 0'])

In [14]:
df_final = pd.merge(df_first, df_second, on='person', how='left')

df_final = pd.merge(df_final, df_thrid, on='person', how='left')

In [15]:
df_final.columns.values

array(['Android', 'BlackBerry', 'Chrome', 'Computer', 'FreeBSD', 'Linux',
       'Mac', 'Other', 'Smartphone', 'Tablet', 'Tizen', 'Ubuntu',
       'Unknown', 'Windows', 'ad campaign hit', 'brand listing',
       'checkout', 'conversion', 'generic listing', 'iOS', 'lead',
       'search engine hit', 'searched products', 'staticpage', 'target',
       'viewed product', 'visited site', 'person',
       'diff_time_days_minnone', 'diff_time_daysnone',
       'diff_time_dayscheckout', 'diff_time_daysconversion',
       'diff_time_daysbrand listing', 'diff_time_dayssearch engine hit',
       'diff_time_start_end', 'model', 'number_visits_model',
       'COUNT(users_logs)', 'avg_time_events'], dtype=object)

Significado de las columnas:

 - Las columnas 'Android', 'BlackBerry', 'Chrome', 'FreeBSD', 'Linux', 'Mac', 'Tizen', 'Ubuntu', 'Unknown', 'Windows','iOS' indican la cantidad de veces que cada usuario ingreso a la pagina de Trocafone utilizando uno de esos esos sistemas operativos. Por ejemplo, si el valor de 'Android' para un usuario es 2 siginfica que ese usuario entro a la pagina dos veces desde un dispositivo con un sistema operativo Android
 
 - Las columnas 'Computer', 'Smartphone', 'Tablet' indican la cantidad de veces que cada usuario ingreso a la pagina de Trocafone utilizando uno de esos dispositivos
 
 - Las columnas 'ad campaign hit', 'brand listing', 'checkout', 'conversion', 'generic listing', 'lead','search engine hit', 'searched products', 'staticpage','viewed product', 'visited site' indican la cantidad de veces que cada usuario realizo cada uno de esos eventos
 
 - La columna 'diff_time_days_minnone' indica la cantidad de tiempo que paso desde el primer evento que realizo el usuario hasta el 2018/06/01
 
 - La columna 'diff_time_daysnone' indica la cantidad de tiempo que paso desde el ultimo evento que el usuario realizo en la pagina hasta el 2018/06/01
 
 - La columna 'diff_time_dayscheckout' indica la cantidad de tiempo que paso desde que el usuario realizo el ultimo checkout hasta el 2018/06/01
 
 - La columna 'diff_time_daysconversion' indica la cantidad de tiempo que paso desde que el usuario realizo la ultima conversion hasta el 2018/06/01
 
 - La columna 'diff_time_daysbrand listing' indica la cantidad de tiempo que paso desde que el usuario realizo la ultima brand listing hasta el 2018/06/01
 
 - La columna 'diff_time_dayssearch engine hit' indica la cantidad de tiempo que paso desde que el usuario realizo el ultimo engine hit hasta el 2018/06/01
 
 - La columna 'diff_time_start_end' indica la cantidad de tiempo que paso entre el primer evento que realizo el usuario en la pagina y el ultimo
 
 - La columna 'model' indica el modelo de celuar que el usuario mas veces visito
 
 - La columna 'number_visits_model' indica cuantas veces el usuario visito el celular que mas visito
 
 - La columna 'COUNT(users_logs)' indica la cantidad de eventos que el usuario realizo en la pagina
 
 - La columna 'avg_time_events' indica el tiempo promedio que paso entre los eventos realizados por cada usuario

## Reemplazamos con zeros los NaNs

In [16]:
df_final.fillna(value=0,inplace=True)

In [17]:
df_final.columns

Index(['Android', 'BlackBerry', 'Chrome', 'Computer', 'FreeBSD', 'Linux',
       'Mac', 'Other', 'Smartphone', 'Tablet', 'Tizen', 'Ubuntu', 'Unknown',
       'Windows', 'ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'iOS', 'lead', 'search engine hit',
       'searched products', 'staticpage', 'target', 'viewed product',
       'visited site', 'person', 'diff_time_days_minnone',
       'diff_time_daysnone', 'diff_time_dayscheckout',
       'diff_time_daysconversion', 'diff_time_daysbrand listing',
       'diff_time_dayssearch engine hit', 'diff_time_start_end', 'model',
       'number_visits_model', 'COUNT(users_logs)', 'avg_time_events'],
      dtype='object')

### Eliminamos las columnas con muy pocas variables o alta corrleación (Windows ~ Computer)

In [18]:
df_final.drop(["BlackBerry","Chrome","FreeBSD","Linux","Mac","Other","Tizen","Ubuntu","Unknown","Windows"],axis=1,inplace=True)

### Transformamos algunas columnas para que su distribución se ajuste a una Chi cuadrado o Normal. Luego discretizamos por cuantiles

In [19]:
df_final.loc[:,"diff_time_daysnone"] = np.log10(df_final["diff_time_daysnone"]+1.)

In [20]:
df_final.loc[:,"diff_time_days_minnone"] = np.log10(df_final.diff_time_days_minnone+1.)

In [21]:
df_final.loc[:,"diff_time_dayssearch engine hit"] = np.log10(df_final["diff_time_dayssearch engine hit"]+1.0)

In [22]:
df_final.loc[:,"COUNT(users_logs)"] = np.log10(df_final["COUNT(users_logs)"]+1.)

In [23]:
df_final.loc[:,"number_visits_model"] = np.log10(1.+df_final.number_visits_model)

In [24]:
df_final.loc[:,"diff_time_daysbrand listing"] = np.log10(01.+df_final["diff_time_daysbrand listing"])

In [25]:
for column in ["diff_time_days_minnone","diff_time_dayssearch engine hit","COUNT(users_logs)","number_visits_model","diff_time_daysbrand listing","diff_time_daysnone"]:
    df_final = discretize(df_final,column)
    

In [26]:
df_final["marca"] = df_final.model.str.split(" ",1,expand=True)[0]
df_final["modelo"] = df_final.model.str.split(" ",1,expand=True)[1]

In [27]:
df_final["marca"].value_counts()

iPhone      8017
Samsung     7663
Motorola    2257
LG           301
Lenovo       146
Sony         108
iPad          36
Asus          31
Quantum       11
Name: marca, dtype: int64

In [28]:
df_final["modelo"].value_counts()

6                               1752
5s                              1639
6S                              1361
Galaxy J5                       1022
7                                703
7 Plus                           613
Galaxy S8                        609
Galaxy S7                        604
Galaxy S7 Edge                   603
Galaxy J7 Prime                  489
Moto G2 3G Dual                  404
Galaxy S6 Edge                   393
Galaxy S6 Flat                   386
5c                               352
6S Plus                          344
Galaxy J7                        337
6 Plus                           337
Moto G4 Plus                     313
SE                               311
Galaxy Gran Prime Duos TV        245
4S                               232
Galaxy S8 Plus                   230
5                                223
Moto G3 4G                       191
Galaxy A5 2017                   185
Moto X Play 4G Dual              174
Moto G5 Plus                     169
G

In [29]:
df_final["is_iphone"] = list(map(lambda x: 1 if x=="iPhone" else 0, df_final["marca"]))
df_final["is_samsung"] = list(map(lambda x: 1 if x=="Samsung" else 0, df_final["marca"]))
df_final["is_motorola"] = list(map(lambda x: 1 if x=="Motorola" else 0, df_final["marca"]))
df_final["is_lg"] = list(map(lambda x: 1 if x=="LG" else 0, df_final["marca"]))
df_final["is_sony"] = list(map(lambda x: 1 if x=="Sony" else 0, df_final["marca"]))
df_final["is_other"] = list(map(lambda x: 1 if (x!="Sony" and x!="LG" and x!="Motorola" and x!="Samsung" and x!="iPhone") else 0, df_final["marca"]))


In [30]:
pop_models = ['6','5s','6S','Galaxy J5','7','7 Plus','Galaxy S8','Galaxy S7','Galaxy S7 Edge','Galaxy J7 Prime','Moto G2 3G Dual','Galaxy S6 Edge','Galaxy S6 Flat','5c','6S Plus','Galaxy J7','6 Plus','Moto G4 Plus','SE','Galaxy Gran Prime Duos TV','4S','Galaxy S8 Plus','5','Moto G3 4G','Galaxy A5 2017','Moto X Play 4G Dual','Moto G5 Plus','Galaxy A7 2017','Moto X2','Galaxy A5']

In [31]:
for model in pop_models:
    df_final = is_object(df_final,"modelo",model)

In [32]:
df_final["is_other_model"] = list(map(lambda x: 1 if x not in pop_models else 0, df_final["modelo"]))

In [33]:
df_final[["marca","modelo","is_other_model"]].head()

,marca,modelo,is_other_model
0,NaN,NaN,1
1,iPhone,7,0
2,iPhone,6S,0
3,Motorola,Moto G2 3G Dual,0
4,Samsung,Galaxy S8 Plus,0


In [34]:
print(len(df_final.columns.values))

df_final.columns.values

86


array(['Android', 'Computer', 'Smartphone', 'Tablet', 'ad campaign hit',
       'brand listing', 'checkout', 'conversion', 'generic listing',
       'iOS', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'target', 'viewed product', 'visited site', 'person',
       'diff_time_dayscheckout', 'diff_time_daysconversion',
       'diff_time_start_end', 'model', 'avg_time_events',
       'diff_time_days_minnone_in_q1', 'diff_time_days_minnone_in_q2',
       'diff_time_days_minnone_in_q3', 'diff_time_days_minnone_in_q4',
       'diff_time_dayssearch engine hit_in_q1',
       'diff_time_dayssearch engine hit_in_q2',
       'diff_time_dayssearch engine hit_in_q3',
       'diff_time_dayssearch engine hit_in_q4', 'COUNT(users_logs)_in_q1',
       'COUNT(users_logs)_in_q2', 'COUNT(users_logs)_in_q3',
       'COUNT(users_logs)_in_q4', 'number_visits_model_in_q1',
       'number_visits_model_in_q2', 'number_visits_model_in_q3',
       'number_visits_model_in_q4', 'diff_time_day

In [35]:
df_final['target'].value_counts()

0.0    18434
1.0      980
Name: target, dtype: int64

### Vamos a seleccionar solo 980 casos que no realizaron una compra para que el dataset quede balanceado

In [36]:
df_final_reduced = df_final[df_final['target']==1]

print(len(df_final_reduced.index))

980


In [37]:
df_final_reduced_0 = df_final[df_final['target']==0].sample(n=980, random_state=1)

print(len(df_final_reduced_0.index))

980


In [38]:
df_final_reduced = pd.concat([df_final_reduced, df_final_reduced_0])

print(len(df_final_reduced.index))

1960


### Obtenemos el dataset de test

In [39]:
df_test = df_final_reduced.sample(frac=0.2, random_state=2)

print(len(df_test.index))


392


In [40]:
df_test.head()

,Android,Computer,Smartphone,Tablet,ad campaign hit,brand listing,checkout,conversion,generic listing,iOS,...,is_Galaxy S8 Plus,is_5,is_Moto G3 4G,is_Galaxy A5 2017,is_Moto X Play 4G Dual,is_Moto G5 Plus,is_Galaxy A7 2017,is_Moto X2,is_Galaxy A5,is_other_model
14935,0.0,3.0,0.0,0.0,3.0,6.0,0.0,0.0,4.0,0.0,...,0,0,0,0,0,0,0,0,0,0
12670,1.0,0.0,1.0,0.0,2.0,2.0,1.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
121,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
17292,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
14172,16.0,3.0,16.0,0.0,8.0,0.0,2.0,1.0,21.0,0.0,...,0,0,0,0,0,0,0,0,0,0


Obtenemos el dataset de "work", a partir del cual vamos a obtener el dataset de training y el de validacion

In [41]:
df_work = df_final_reduced.loc[~df_final_reduced.index.isin(df_test.index)]

print(len(df_work.index))

1568


In [42]:
df_work = df_work.sample(frac=1).reset_index(drop=True)

In [46]:
print(len(df_work.index))

df_work.head(3)

1568


,Android,Computer,Smartphone,Tablet,ad campaign hit,brand listing,checkout,conversion,generic listing,iOS,...,is_Galaxy S8 Plus,is_5,is_Moto G3 4G,is_Galaxy A5 2017,is_Moto X Play 4G Dual,is_Moto G5 Plus,is_Galaxy A7 2017,is_Moto X2,is_Galaxy A5,is_other_model
0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


### Sacamos columnas que no son numericas

In [47]:
df_work = df_work.drop(columns=['person', 'model', 'marca', 'modelo'])

### Agregamos una columna al dataset que indica a que fold pertenece

In [49]:
def generate_number_fold(dataframe, number_folds):
    
    number_rows = len(dataframe.index)
    
    rows_per_fold = round(number_rows/number_folds)
    
    array_number_folds = []
    
    index = 1
    
    while index < (number_folds + 1):
        
        if (number_folds - index) == 1:
            
            array_numbers = [index] * (number_rows - rows_per_fold*index)
        
            array_number_folds = array_number_folds + array_numbers
            
        else:
            
            array_numbers = [index] * rows_per_fold

            array_number_folds = array_number_folds + array_numbers
            
        index = index + 1

    dataframe['folds'] = array_number_folds
    
    return dataframe
        

### Filtramos por los folds que generamos con generate_number_fold() y vamos calculando las metricas que necesitamos tanto para training como para validation

In [64]:
def applied_cross_validation_dt(dataframe, number_folds, depth, criterion):
    
    df_folds = generate_number_fold(dataframe, number_folds)
    
    index = 1
    
    array_accuracy_training = []
    
    array_accuracy_val = []
    
    array_auc_training = []
    
    array_auc_val = []
    
    while index < (number_folds + 1):
        
        df_specific_fold = df_folds[df_folds['folds']==index].drop(columns=['folds'])
        
        X = df_specific_fold.drop(columns=['target'])

        y = df_specific_fold['target']
        
        train_X, test_X, train_y, test_y = train_test_split(X, y, 
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=123)
        
        dtree=DecisionTreeClassifier(criterion=criterion, max_depth=depth)
        
        dtree.fit(train_X, train_y)
        
        predictions_val = dtree.predict(test_X)
        predictions_train = dtree.predict(train_X)
        
        proba_val = dtree.predict_proba(test_X)
        proba_train = dtree.predict_proba(train_X)
        
        acc_val = accuracy_score(test_y, predictions_val)
        acc_train = accuracy_score(train_y, predictions_train)
        
        auc_val = roc_auc_score(test_y, proba_val[:,1])
        auc_train = roc_auc_score(train_y, proba_train[:,1])
        
        array_accuracy_val.append(acc_val)
        
        array_accuracy_training.append(acc_train)
        
        array_auc_training.append(auc_val)
    
        array_auc_val.append(auc_train)
        
        index = index + 1
    
    return array_accuracy_training, array_accuracy_val, array_auc_training, array_auc_val

In [71]:
applied_cross_validation_dt(df_work, 5, 3, 'gini')

([0.7899543378995434,
  0.8310502283105022,
  0.7990867579908676,
  0.8486238532110092,
  0.8036529680365296],
 [0.8210526315789474,
  0.7789473684210526,
  0.8,
  0.7340425531914894,
  0.6947368421052632],
 [0.8482222222222221,
  0.7772606382978724,
  0.8109485815602837,
  0.7591575091575091,
  0.7034313725490197],
 [0.8720706394375627,
  0.904516507457684,
  0.8673401235598597,
  0.8800252844500632,
  0.8559024878944732])

In [68]:
def generate_table(acc_train, acc_val, auc_train, auc_val, criterion, depth, number_folds):
    
    d = {'acc_train': acc_train, 'acc_val': acc_val, 'auc_train': auc_train, 'auc_val': auc_val}
    
    df = pd.DataFrame(data=d)
    
    df['criterion'] = [criterion] * number_folds
    
    df['max_depth'] = [depth] * number_folds
    
    return df

In [74]:
acc_train_gini = applied_cross_validation_dt(df_work, 5, 3, 'gini')[0]

acc_val_gini = applied_cross_validation_dt(df_work, 5, 3, 'gini')[1]

auc_train_gini = applied_cross_validation_dt(df_work, 5, 3, 'gini')[2]

auc_val_gini = applied_cross_validation_dt(df_work, 5, 3, 'gini')[3]

df_gini_3 = generate_table(acc_train_gini, acc_val_gini, auc_train_gini, auc_val_gini, 'Gini', 3, 5)

df_gini_3 = df_gini_3.append(df_gini_3.mean(numeric_only=True), ignore_index=True)

df_gini_3

,acc_train,acc_val,auc_train,auc_val,criterion,max_depth
0,0.789954,0.821053,0.887778,0.872071,Gini,3.0
1,0.831050,0.778947,0.777261,0.904517,Gini,3.0
2,0.799087,0.800000,0.810949,0.867340,Gini,3.0
3,0.848624,0.723404,0.759158,0.880025,Gini,3.0
4,0.803653,0.694737,0.703431,0.855902,Gini,3.0
5,0.814474,0.763628,0.787715,0.875971,NaN,3.0


In [ ]:
def highlight_total_row(s):
    if s.B > 1.0:
        return ['background-color: yellow']*5
 
df.style.apply(highlight_greaterthan_1, axis=1)

In [72]:
acc_train_gini = applied_cross_validation_dt(df_work, 5, 6, 'gini')[0]

acc_val_gini = applied_cross_validation_dt(df_work, 5, 6, 'gini')[1]

auc_train_gini = applied_cross_validation_dt(df_work, 5, 6, 'gini')[2]

auc_val_gini = applied_cross_validation_dt(df_work, 5, 6, 'gini')[3]

generate_table(acc_train_gini, acc_val_gini, auc_train_gini, auc_val_gini, 'Gini', 6, 5)

,acc_train,acc_val,auc_train,auc_val,criterion,max_depth
0,0.894977,0.810526,0.749778,0.962086,Gini,6
1,0.936073,0.684211,0.698582,0.981691,Gini,6
2,0.949772,0.684211,0.625000,0.990858,Gini,6
3,0.935780,0.670213,0.662317,0.971260,Gini,6
4,0.899543,0.705263,0.672460,0.954667,Gini,6
